# Final Project Submission

Please fill out:
* Student name: **Brian Bentson**
* Student pace: self paced / part time / full time: **Full Time**
* Scheduled project review date/time: 
* Instructor name: **James Irving**
* Blog post URL:
* Video of 5-min Non-Technical Presentation:

# Table of Contents

*Click to jump to matching Markdown Header.*<br><br>
 
- **[Introduction](#Introduction)<br>**
- **[OBTAIN](#Obtain)**<br>
- **[SCRUB](#Scrub)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

# Introduction

## Business Statement

One primary roles of a realtor is to provide their clients with a good initial estimate for a listing price. The listing price can't be too high, or else it may scare off potential buyers, and the price cannot be too low because then the home seller will sacrifice potential profit from their equity. Therefore, if a model can be built which predicts the appropriate listing price, a home seller can be confident that they will sell their house at a favorable price.

## Analysis Methodology

I will be analyzing historic home sales from King County, Washington in order to see which factors affect home price and how a model can be built to predict good estimates for home listing prices.

# Obtain

## Import Packages

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [146]:
pd.set_option("display.max_columns", 22)

## Global Functions

## Import Data into Pandas

I will be importing a csv dataset which provides me with the information necessary to begin the analysis.

In [147]:
#import the dataset from local csv
df_original = pd.read_csv('Data/kc_house_data.csv')
df_original

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,3,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,3,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,3,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


## Investigate Data

In [148]:
#column names
df_original.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [149]:
#view df info to inspect data types
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

> **Observations on Data**
> - `waterfront` values should be updated to categorical
> - `yr_renovated` values should be updated to categorical
> - `sqft_basement` data type should be changed to integer

In [150]:
#check for null values
df_original.isna().sum()/len(df_original)*100

id                0.000000
date              0.000000
price             0.000000
bedrooms          0.000000
bathrooms         0.000000
sqft_living       0.000000
sqft_lot          0.000000
floors            0.000000
waterfront       11.001528
view              0.291707
condition         0.000000
grade             0.000000
sqft_above        0.000000
sqft_basement     0.000000
yr_built          0.000000
yr_renovated     17.789508
zipcode           0.000000
lat               0.000000
long              0.000000
sqft_living15     0.000000
sqft_lot15        0.000000
dtype: float64

> **Observations on Data**
> - `waterfront` has 11% null values which can be dealt with when changing to a categorical data type
> - `view` should be inspected for a potential column drop
> - `yr_renovated` has 18% null values which can be dealt with when changing to a categorical data type
> - All other columns have 0 nulls

# Scrub

I will make a new dataframe which is a copy of the `df_original` dataframe to begin making changes. 

In [151]:
#create a copy of the original dataframe
df_scrub = df_original.copy()
df_scrub

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,3,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,3,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,3,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


## Create New Columns

### Basement Column

In the dataset we have 3 related columns:
- `sqft_above`
- `sqft_basement`
- `sqft_living`

These columns are related in that `sqft_living` equals `sqft_above` plus `sqft_basement`. I do not think the square footage of the basement is as important as just knowing that a house has one. Therefore, I will create a new column which shows whether or not a house has a basement.

In [152]:
#investigate values in sqft_basement
df_scrub['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
2730.0        1
4130.0        1
906.0         1
2240.0        1
556.0         1
Name: sqft_basement, Length: 304, dtype: int64

> **'?' impedes the ability to create a new column. Will drop the 454 rows, which is 2% of the data**

In [154]:
#drop rows with '?' value in sqft_basement column
df_scrub.drop(df_scrub.loc[df_scrub['sqft_basement'] == '?'].index,inplace=True)
len(df_scrub)

21143

In [162]:
#prove that these columns are related
df_scrub['sqft_basement'] = df_scrub['sqft_basement'].astype(float).astype(int)
sqft = df_scrub[['sqft_living', 'sqft_above', 'sqft_basement']]
(sqft['sqft_above'] + sqft['sqft_basement'] == sqft['sqft_living']).value_counts()
df_scrub['sqft_basement'].dtype

dtype('int64')

In [163]:
#check the rows have been dropped
df_scrub.loc[df_original['sqft_basement'] == '?']['sqft_basement'].count()

0

In [164]:
#create new column for basement
df_scrub['basement'] = np.where(df_scrub['sqft_basement'] >0, 'Yes','No')
df_scrub

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,basement
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0,1955,0.0,98178,47.5112,-122.257,1340,5650,No
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400,1951,1991.0,98125,47.7210,-122.319,1690,7639,Yes
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0,1933,NaN,98028,47.7379,-122.233,2720,8062,No
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910,1965,0.0,98136,47.5208,-122.393,1360,5000,Yes
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0,1987,0.0,98074,47.6168,-122.045,1800,7503,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,3,8,1530,0,2009,0.0,98103,47.6993,-122.346,1530,1509,No
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,3,8,2310,0,2014,0.0,98146,47.5107,-122.362,1830,7200,No
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0,2009,0.0,98144,47.5944,-122.299,1020,2007,No
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,3,8,1600,0,2004,0.0,98027,47.5345,-122.069,1410,1287,No


### Month Sold Column

I want to extract the month sold to investigate if sale timing has any affect on housing prices

In [105]:
#create new month sold column
df_scrub['date'] = pd.to_datetime(df_scrub['date'])
df_scrub['month_sold'] = df_scrub['date'].dt.month
df_scrub.loc[:3,['date','month_sold']]

,date,month_sold
0,2014-10-13,10
1,2014-12-09,12
2,2015-02-25,2
3,2014-12-09,12


## Categorize Columns

## Change Data Types

In [59]:
#change datatype of sqft_basement to int 
df_original['sqft_basement'].astype(float).astype(int)

0          0
1        400
2          0
3        910
4          0
        ... 
21592      0
21593      0
21594      0
21595      0
21596      0
Name: sqft_basement, Length: 21143, dtype: int64

## Column Drop

In [106]:
#look at columns
df_scrub.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'basement', 'month_sold'],
      dtype='object')

In [107]:
#look at view column
df_scrub['view'].value_counts()

KeyError: 'view'

> **The `view` column does not seem to be important to keep going forward.**

In [65]:
#drop the view column
df_original.drop(columns='view', inplace=True)
df_original.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [80]:
#drop the sqft_basement column
df_original.drop(columns='view', inplace=True)
df_original.columns

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
249,3797001895,4/22/2015,481000.0,3,1.75,1560,3000,1.0,0.0,4,6,770,790,1918,0.0,98103,47.6846,-122.345,1390,3000
880,2349300060,2/12/2015,200000.0,4,2.00,1920,4822,1.0,0.0,2,6,920,1000,1914,0.0,98136,47.5507,-122.381,1120,4822
991,1455600030,1/8/2015,645000.0,4,2.00,2780,11583,1.0,0.0,3,8,1190,1590,1955,0.0,98125,47.7293,-122.284,2580,10241
1200,1231000510,9/22/2014,263000.0,3,1.75,1490,3800,1.0,0.0,3,6,700,790,1913,0.0,98118,47.5554,-122.270,2180,4000
1201,1231000510,5/4/2015,510000.0,3,1.75,1490,3800,1.0,0.0,3,6,700,790,1913,0.0,98118,47.5554,-122.270,2180,4000
1206,3885804305,9/11/2014,949000.0,4,1.75,2490,7834,1.0,0.0,4,8,1240,1250,1958,0.0,98033,47.6851,-122.209,3210,7834
1603,925049360,4/28/2015,512000.0,2,2.00,1270,3881,1.0,0.0,4,6,610,660,1926,0.0,98105,47.6694,-122.298,1370,5000
1735,5402100045,3/11/2015,189950.0,4,2.00,1910,4225,1.0,0.0,4,6,910,1000,1919,NaN,98001,47.3084,-122.234,1060,4800
2262,4037400295,7/30/2014,618000.0,4,2.25,2530,8736,1.0,0.0,4,7,1210,1320,1958,0.0,98008,47.6049,-122.126,1720,8500
2527,3126049094,11/12/2014,392450.0,4,2.00,2195,2681,1.0,0.0,3,7,1060,1135,1912,0.0,98103,47.6965,-122.342,1710,1280
